In [8]:
# Celda 1: Instalar dependencias
!pip install --quiet deap numpy azure-cosmos
!pip install --quiet deap azure-cosmos numpy
# Celda 1: Instalar dependencias (deap, numpy, azure-cosmos)
!pip install --quiet deap numpy azure-cosmos
import sys
!{sys.executable} -m pip install deap numpy azure-cosmos --quiet

import numpy as np
from deap import base, creator, tools, algorithms
from azure.cosmos import CosmosClient

print("Librerías importadas correctamente.")



Librerías importadas correctamente.


In [9]:
# Celda 2: Importar librerías y conectar a Cosmos DB

import numpy as np
from deap import base, creator, tools, algorithms
from azure.cosmos import CosmosClient
import json
import os

# Leer endpoint y clave de variables de entorno (puedes definirlas en Azure ML)
endpoint = os.getenv("COSMOS_ENDPOINT", "https://infoitems.documents.azure.com:443/")
key = os.getenv("COSMOS_KEY", "Fi8suofAtOL8xMP3qsoMzTsHqwUuNvUQF6c9oeJnkPTm387SyM0CyubYJQnu7dNaSd4bl37PAXgbACDbntPsgQ==")

client = CosmosClient(endpoint, key)
database = client.get_database_client("colisiones")
container = database.get_container_client("colisiones")
query = "SELECT TOP 1 * FROM c ORDER BY c.timestamp DESC"
items = list(container.query_items(query, enable_cross_partition_query=True))
prediction = items[0]['predictions'][0] if items else {'confidence': 0.5}

print(f"Confianza predicción obtenida: {prediction['confidence']}")


Confianza predicción obtenida: 0.75


In [10]:
# Celda 3: Definir algoritmo genético y función objetivo

def evaluate_maneuver(individual):
    delta_v = individual[0]  # m/s
    fuel_cost = abs(delta_v) * 10
    collision_risk = max(0, 100 - delta_v * 50 * prediction['confidence'])
    return fuel_cost, collision_risk

# Crear clases de fitness e individuo
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))  # Minimizar fuel y riesgo
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, -10, 10)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_maneuver)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)


In [11]:
# Celda 4: Ejecutar el algoritmo genético

pop = toolbox.population(n=50)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)

pop, log = algorithms.eaSimple(
    pop, toolbox,
    cxpb=0.5, mutpb=0.2,
    ngen=40, stats=stats,
    halloffame=hof,
    verbose=True
)

best_maneuver = hof[0][0]
resultado = {
    "best_delta_v": best_maneuver,
    "fuel_cost": abs(best_maneuver) * 10,
    "collision_risk": max(0, 100 - best_maneuver * 50 * prediction['confidence'])
}

print(json.dumps(resultado, indent=4))


gen	nevals	avg    	min
0  	50    	106.476	0  
1  	32    	75.045 	0  
2  	34    	56.6379	0.55948
3  	35    	50.1357	0.501437
4  	25    	50.0954	0.265045
5  	29    	49.8437	0.00918329
6  	28    	50.2339	0.00918329
7  	30    	50.4672	0.00918329
8  	28    	50.0471	0.000878385
9  	26    	49.8007	0.000878385
10 	30    	51.4082	0.000878385
11 	32    	49.9109	0.000306139
12 	26    	51.2271	7.56872e-05
13 	26    	50.0025	7.56872e-05
14 	33    	50.1461	1.86609e-06
15 	32    	50.3709	1.86609e-06
16 	29    	50.4755	1.6036e-06 
17 	30    	50.1307	9.19984e-07
18 	24    	50     	5.00751e-07
19 	32    	50.6414	3.52736e-08
20 	31    	50.0553	2.45465e-08
21 	28    	50.7565	7.34254e-09
22 	34    	50.625 	4.0657e-09 
23 	32    	49.5281	4.0657e-09 
24 	31    	50     	1.63994e-09
25 	29    	49.8344	9.67425e-10
26 	36    	51.6882	9.67425e-10
27 	31    	50.5264	1.65555e-12
28 	28    	50.9334	1.65555e-12
29 	32    	51.1574	1.65555e-12
30 	31    	49.9516	1.65555e-12
31 	28    	50.1542	3.42185e-13
32 	25    	49.

In [1]:
import numpy as np
from deap import base, creator, tools, algorithms
from azure.cosmos import CosmosClient
import json
import os

# Leer endpoint y clave de variables de entorno para seguridad
endpoint = os.getenv("COSMOS_ENDPOINT", "https://infoitems.documents.azure.com:443/")
key = os.getenv("COSMOS_KEY", "Fi8suofAtOL8xMP3qsoMzTsHqwUuNvUQF6c9oeJnkPTm387SyM0CyubYJQnu7dNaSd4bl37PAXgbACDbntPsgQ==")

# Conexión a Cosmos DB
client = CosmosClient(endpoint, key)
database = client.get_database_client("colisiones")
container = database.get_container_client("colisiones")
query = "SELECT TOP 1 * FROM c ORDER BY c.timestamp DESC"
items = list(container.query_items(query, enable_cross_partition_query=True))
prediction = items[0]['predictions'][0] if items else {'confidence': 0.5}

def evaluate_maneuver(individual):
    delta_v = individual[0]  # m/s
    fuel_cost = abs(delta_v) * 10
    collision_risk = max(0, 100 - delta_v * 50 * prediction['confidence'])
    return fuel_cost, collision_risk

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, -10, 10)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_maneuver)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

pop = toolbox.population(n=50)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=stats, halloffame=hof)

best_maneuver = hof[0][0]
print(json.dumps({
    "best_delta_v": best_maneuver,
    "fuel_cost": abs(best_maneuver) * 10,
    "collision_risk": max(0, 100 - best_maneuver * 50 * prediction['confidence'])
}))


ModuleNotFoundError: No module named 'deap'